In [1]:
import template
import config
import json
import random

In [2]:
print(template.CTC_templates)

['判断临床试验筛选标准的类型：\n[INPUT_TEXT]\n选项：[LIST_LABELS]\n答：', '确定试验筛选标准的类型：\n[INPUT_TEXT]\n类型选项：[LIST_LABELS]\n答：', '[INPUT_TEXT]\n这句话是什么临床试验筛选标准类型？\n类型选项：[LIST_LABELS]\n答：', '[INPUT_TEXT]\n是什么临床试验筛选标准类型？\n选项：[LIST_LABELS]\n答：', '请问是什么类型？\n[INPUT_TEXT]\n临床试验筛选标准选项：[LIST_LABELS]\n答：']


In [8]:
def change_data(file,task_name,output_name):
    formatted_data_list = []
    with open(file, 'r') as json_file:
        data = json.load(json_file)
    if task_name=="CTC":
        for item in data:
            input_template = random.choice(template.CTC_templates)
            formatted_item = {
                "input": input_template.replace("[INPUT_TEXT]", item["text"]).replace("[LIST_LABELS]", ",".join(config.QQR_category_mapping.values())),
                "target": config.QQR_category_mapping[item["label"]],
                "answer_choices": list(config.QQR_category_mapping.values()),
                "task_type": "cls",
                "task_dataset": "CHIP-CTC",
                "sample_id": item["id"]
            }
            formatted_data_list.append(formatted_item)
        # 将格式化后的数据以一行一个 JSON 对象的格式写入文件
    elif task_name=="QQR":
        for item in data:
            input_template = random.choice(template.QQR_templates)
            formatted_item = {
                "input": input_template.replace("[INPUT_TEXT1]", item["query1"]).replace("INPUT_TEXT2",item["query2"]).replace("[LIST_LABELS]", ",".join(config.QQR_choice)),
                "target": item["label"],
                "answer_choices": list(config.QQR_choice),
                "task_type": "nli",
                "task_dataset": "CHIP-QQR",
                "sample_id": item["id"]
            }
            formatted_data_list.append(formatted_item)
        # 将格式化后的数据以一行一个 JSON 对象的格式写入文件
    elif task_name=="CMeIE":
        count = 1
        for item in data:
            item = json.loads(item)
            input_template = random.choice(template.CMeIE_templates)
            pre_list = []
            sub_list = []
            ob_list = []
            target = []
            tar_list = {"pre_list":pre_list, "sub_list":sub_list, "ob_list":ob_list,"target":target}
            for spo in item["spo_list"]:
                tar_list["pre_list"].append(spo["predicate"])
                tar_list["sub_list"].append(spo["subject"])
                tar_list["ob_list"].append(spo["object"]["@value"])
                tar_list['target'].append("具有"+spo["predicate"]+"关系的头尾体如下：头实体为"+spo["subject"]+",尾实体为"+spo["object"]["@value"])
            formatted_item = {
                "input": input_template.replace("[INPUT_TEXT]", item["text"]).replace("[LIST_LABELS]", ",".join(tar_list["pre_list"])),
                "target": "\n".join(tar_list["target"]),
                "answer_choices": tar_list["pre_list"],
                "task_type": "spo_generation",
                "task_dataset": "CMeIE",
                "sample_id": "CMeIE"+str(count)
            }
            formatted_data_list.append(formatted_item)
            count += 1
    elif task_name=="QIC":
        for item in data:
            input_template = random.choice(template.QIC_templates)
            formatted_item = {
                "input": input_template.replace("[INPUT_TEXT]", item["query"]).replace("[LIST_LABELS]", ",".join(config.QIC_categories.values())),
                "target": item["label"],
                "answer_choices": list(config.QIC_categories.values()),
                "task_type": "cls",
                "task_dataset": "CKUAKE-QIC",
                "sample_id": item["id"]
            }
            formatted_data_list.append(formatted_item)
    elif task_name=="CDN":
        n = 0
        formatted_data_list = []
        for item in data:
            n = n + 1
            input_template = random.choice(template.CDN_templates)
            format_item = {
                #"input": input_template.replace("[INPUT_TEXT]", item["text"]).replace("[LIST_LABELS]", "，".join(item["normalized_result"].split("##"))),
                "input": input_template.replace("[INPUT_TEXT]", item["text"]).replace("[LIST_LABELS]", ""),
                "target": "，".join(item["normalized_result"].split("##")),
                "answer_choices": [""],
                "task_type": "normalization",
                "task_dataset": "CHIP-CDN",
                "sample_id": n
            }
            formatted_data_list.append(format_item)
    elif task_name=="STS":
        formatted_data_list = []
        for item in data:
            input_template = random.choice(template.STS_templates)
            format_item = {
                "input": input_template.replace("[INPUT_TEXT_1]", item["text1"]).replace("[INPUT_TEXT_2]", item["text2"]),
                "target":item["label"],
                "answer_choices": ["相同", "不同"],
                "task_type": "cls",
                "task_dataset": "CHIP-STS",
                "sample_id": item["id"]
            }
            formatted_data_list.append(format_item)
    elif task_name=="CMeEE":
        formatted_data_list = []
        index=0
        for item in data:
            input_template = random.choice(template.CMeEE_templates)
            entities_list = []
            
            for key in config.CMeEE_category_mapping:
                for entity in item["entities"]:
                    if entity["type"] == key:
                        entities_list.append(config.CMeEE_category_mapping[key]+"实体："+entity["entity"])
            index=index+1
            formatted_item = {
                "input": input_template.replace("[INPUT_TEXT]", item["text"]).replace("[LIST_LABELS]", ",".join(config.CMeEE_category_mapping.values())),
                "target": "上述句子中的实体包含：\n"+'\n'.join(entities_list),
                "answer_choices": list(config.CMeEE_category_mapping.values()),
                "task_type": "ner",
                "task_dataset": "CMeEE-V2",
                "sample_id": index
            }
            formatted_data_list.append(formatted_item)


    with open(output_name+'.json', 'w') as output_file:
            for formatted_item in formatted_data_list:
                json.dump(formatted_item, output_file, ensure_ascii=False)
                output_file.write('\n')  # 在 JSON 对象之间换行分隔


In [9]:
change_data("KUAKE-QQR_test.json","QQR","formatted_data")